In [1]:
#!/usr/local/bin/python

import os, sys
import json
import numpy as np
import matplotlib.pyplot as plt
import chainer
from chainer import cuda
from matplotlib import animation
from optparse import OptionParser

from elecpy.util.cmap_bipolar import bipolar
import elecpy.elecpy as elp

from matplotlib import animation, rc
from IPython.display import HTML

import time

import json

%matplotlib inline

pyculib:1: ImportWarning: Numba version too old; expecting 0.33.0


In [2]:
from elecpy.cell.test_ import test

In [4]:
shape = (1,1)

In [10]:
with open ('./elecpy/temp/pacing_params.json','r') as f:
    pacing_params = json.load(f)

In [17]:
pacing_params['cell_type'] = 'courtmanche'
pacing_params

{u'cell_param': {},
 u'cell_type': 'courtmanche',
 u'stim': {u'amplitude': -80.0,
  u'duration': 0.5,
  u'interval': 400.0,
  u'start': 0.0,
  u'train': 20},
 u'time': {u'dt': 0.005, u'log_cnt': 200}}

In [54]:
from courtmanche.model import model as courtmanche
from luorudy.model import model as luorudy
model_l = luorudy(shape)
model_c = courtmanche(shape)

In [64]:
model_l.update()

In [55]:
model_c.update()

In [82]:
string_args = ''
string_rets = ''

for i, key in enumerate(model_l.params) :
    string_args += 'T {0}, '.format(key)
    string_rets += 'T _{0}, '.format(key)
string_args = string_args.rstrip(', ')
string_rets = string_rets.rstrip(', ')

In [83]:
open(model_l.elemwise_path).read().format(**model_l.const)

'int _maskPosi;\nint _maskNega;\nint _maskPosiPosi;\nint _maskPosiNega;\nint _maskA;\nint _maskA_bar;\nint _maskB;\nint _maskB_bar;\n\n_v = v;\n\n// voltage trapping to avoid error\n_maskPosi =  ( _v < 0.00001 ) * 1;\n_maskPosi *= ( _v >= 0 ) * 1;\n_maskNega = ( _maskPosi == 0 ) * 1;\n_v = _maskPosi * 0.00001 + _maskNega * _v;\n_maskPosi =  ( _v > -0.00001 ) * 1;\n_maskPosi *= ( _v <= 0 ) * 1;\n_maskNega = ( _maskPosi == 0 ) * 1;\n_v = _maskPosi * -0.00001 + _maskNega * _v;\n\n_maskPosi =  ( _v < -30.0+0.001 ) * 1;\n_maskPosi *= ( _v >= -30.0 ) * 1;\n_maskNega = ( _maskPosi == 0 ) * 1;\n_v = _maskPosi * (-30.0+0.001)+ _maskNega * _v;\n_maskPosi =  ( _v > -30.0 -0.001 ) * 1;\n_maskPosi *= ( _v <= -30.0 ) * 1;\n_maskNega = ( _maskPosi == 0 ) * 1;\n_v = _maskPosi * (-30.0-0.001) + _maskNega * _v;\n\n// comp_ina ()\ndouble _shift_inf_m = 0.8*(temp-310.0);\ndouble _shift_inf_hj = 0.7*(temp-310.0);\n\ndouble _ena = ((8314*temp)/96485)*log(130.0/nai);\n_maskPosi = (_v >= -95.0 + _shift_inf_m)

In [34]:
string_args = ''
string_rets = ''

for i, key in enumerate(model_c.params) :
    string_args += 'T {0}, '.format(key)
    string_rets += 'T _{0}, '.format(key)
string_args = string_args.rstrip(', ')
string_rets = string_rets.rstrip(', ')

In [52]:
elemwise=open(model_c.elemwise_path).read().format(**model_c.const)

In [53]:
cuda.elementwise(string_args,
                 string_rets,
                 elemwise,
                 'luorudy')(*model_c.state)

(array([[-81.91649124]]),
 array([[ 0.01]]),
 array([[ 0.]]),
 array([[ 11.19995156]]),
 array([[ 138.99999448]]),
 array([[ 0.00010197]]),
 array([[ -2.66421772e-05]]),
 array([[-0.09898967]]),
 array([[ 0.34829239]]),
 array([[ 0.25756643]]),
 array([[-0.10733012]]),
 array([[  6.70592743e-06]]),
 array([[ 0.00036043]]),
 array([[  3.70461483e-05]]),
 array([[  4.89992117e-09]]),
 array([[ 0.02617063]]),
 array([[-0.00192119]]),
 array([[ 0.0029022]]),
 array([[ 0.96500013]]),
 array([[ 0.97799982]]),
 array([[ 0.00013698]]),
 array([[ 0.99983699]]),
 array([[ 0.0187]]),
 array([[  3.28999978e-05]]),
 array([[ 0.03040001]]),
 array([[ 0.99900019]]),
 array([[ 0.00495998]]),
 array([[ 0.999]]),
 array([[ 0.77499999]]),
 array([[ 0.]]),
 array([[ 1.49]]),
 array([[ 1.49000002]]),
 array([[ 0.01186047]]),
 array([[ 0.00205479]]),
 array([[  3.33439498e-112]]),
 array([[ 1.]]),
 array([[ 0.99900869]]),
 array([[ 0.02540094]]))